In [ ]:
#predict in tensor format

import numpy as np
from PIL import Image
import torch

# Assuming you've loaded the model as before:
model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5/runs/train-seg/exp3/weights/best.pt')

# Load image using PIL
img_path = 'sat-t1-cropped.png'
img_pil = Image.open(img_path)

# Convert PIL Image to a tensor
img_tensor = torch.from_numpy(np.array(img_pil)).permute(2,0,1).float() / 255.0  # Normalize to [0, 1]
img_tensor = img_tensor.unsqueeze(0)  # Add a batch dimension

# Inference
results = model(img_tensor)

# Results
print(type(results))
print(results)


In [5]:
import torch
from pathlib import Path
import json
from PIL import Image
import numpy as np

model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5/runs/train-seg/exp3/weights/best.pt')


class YOLOv5Model:
    def __init__(self, model_path):
        # Load the model
        self.model = torch.load(model_path, map_location='cpu')['model'].float().eval()

    def predict(self, image_tensor):
        # Inference
        with torch.no_grad():
            results = self.model(image_tensor.unsqueeze(0))
        return results

    def to_json(self, results):
        # Extracting detection results
        labels, cords = results[0][0][:, -1].int().tolist(), results[0][0][:, :-1].tolist()
        json_output = [{"label": label, "coordinates": cord} for label, cord in zip(labels, cords)]
        return json_output

if __name__ == "__main__":
    MODEL_PATH = Path("yolov5/runs/train-seg/exp3/weights/best.pt")
    IMG_PATH = 'sat-image-p3.png'
    OUTPUT_JSON = 'sat-image-p3v1.json'
    OUTPUT_IMG = 'sat-image-p3v1.jpg'
    
    yolo_model = YOLOv5Model(MODEL_PATH)
    
    # Load the image and convert to numpy array
    img_pil = Image.open(IMG_PATH)
    img_np = np.array(img_pil)
    img_tensor = torch.from_numpy(img_np).permute(2, 0, 1).float() / 255.0
    
    predictions = yolo_model.predict(img_tensor)
    
    with open(OUTPUT_JSON, 'w') as json_file:
        json.dump(yolo_model.to_json(predictions), json_file, indent=4)
    


Using cache found in /Users/danielyu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-23 Python-3.11.5 torch-2.1.0 CPU

Fusing layers... 
Model summary: 165 layers, 7398422 parameters, 0 gradients, 25.7 GFLOPs
WARNING ⚠️ YOLOv5 SegmentationModel is not yet AutoShape compatible. You will not be able to run inference with this model.


In [9]:
!python3 yolov5/segment/predict.py --hide-labels --save-txt --line-thickness 1 --data Roofs-1/data.yaml --weights yolov5/runs/train-seg/exp3/weights/best.pt --img 640 --conf 0.1 --source sat-image-p3.png 

segment/predict: weights=['yolov5/runs/train-seg/exp3/weights/best.pt'], source=sat-image-p3.png, data=Roofs-1/data.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/predict-seg, name=exp, exist_ok=False, line_thickness=1, hide_labels=True, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
YOLOv5 🚀 v7.0-227-ge4df1ec Python-3.11.5 torch-2.1.0 CPU

Fusing layers... 
Model summary: 165 layers, 7398422 parameters, 0 gradients, 25.7 GFLOPs
image 1/1 /Users/danielyu/heatapp/sat-image-p3.png: 640x640 73 houses, 115.5ms
Speed: 0.8ms pre-process, 115.5ms inference, 1.2ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/predict-seg/exp3
1 labels saved to yolov5/runs/predict-seg/exp3/labels


In [ ]:
from IPython.display import display, Image

img = Image(filename='yolov5/runs/predict-seg/exp3/sat-image-p3.png', width=640)
display(img)
